In [8]:
mnist_models_path = '../output/mnist-models.csv'
mnist_fgm_results_path = '../output/mnist-fgm-results.json'
mnist_cw_results_path = '../output/mnist-cw-results.json'
mnist_bim_results_path = '../output/mnist-bim-results.json'

In [9]:
import pandas as pd
import numpy as np
from collections import OrderedDict

In [3]:
mnist_models = pd.read_csv(mnist_models_path)

In [20]:
def process(df):
    df['accuracy'] = df.accuracies.apply(np.mean)
    df['attack_type'] = df.apply(
        lambda r: ('%s(L-%s,eps=%.1f)' % (r['attack_type'], r['ord'], r['epsilon'])
                   if 'epsilon' in r else r['attack_type']), axis=1)
    # in case there're duplicates
#     df = df.groupby(['model_name', 'attack_type']).agg({'accuracy': np.mean}).reset_index()
    return df

In [38]:
mnist_fgm_results = process(pd.read_json(mnist_fgm_results_path))
# mnist_cw_results = pd.read_json(mnist_cw_results_path)
mnist_cw_results = pd.read_csv('../output/mnist-cnw-results.csv') # wait for newer results
mnist_bim_results = process(pd.read_json(mnist_bim_results_path))

In [49]:
mnist_results = pd.concat([mnist_fgm_results, mnist_bim_results, mnist_cw_results], sort=False)

In [50]:
mnist_results.model_name.drop_duplicates().values

array(['cnn-mnist-relu', 'cnn-mnist-relu-maxfit_l1_01_05',
       'cnn-mnist-relu-maxfit_l2_01_05',
       'cnn-mnist-relu-maxmargin_l1_01_05',
       'cnn-mnist-relu-maxmargin_l2_01_05', 'cnn-mnist-relog',
       'cnn-mnist-relog-more-neurons', 'cnn-mnist-relog-maxout_2',
       'cnn-mnist-relog-minmaxout_4_2',
       'cnn-mnist-relog-minmaxout_4_2-sigmoid_out',
       'cnn-mnist-relog-spherical-minmaxout_4_2-sigmoid_out',
       'cnn-mnist-relog-spherical-minmaxout_4_2-sigmoid_out-max_fit_l1_1',
       'cnn-mnist-relog-spherical-minmaxout_4_2-sigmoid_out-max_fit_l1_1-scrambling',
       'cnn-mnist-relu-kernel',
       'cnn-mnist-relog-minmaxout_4_2-sigmoid_out-max_margin'],
      dtype=object)

In [86]:
def format_table(formated_names, model_order, attack_order):
    df = mnist_results[mnist_results.model_name.isin(formated_names) &
                       mnist_results.attack_type.isin(attack_order)].copy()
    df = df[~df.attack_type.isin(ignored_attacks)]
    df['accuracy'] = df.accuracy.apply(lambda v: '%.2f' % v if type(v) == float else '-')
    df = df.pivot(index='model_name', values='accuracy', columns='attack_type')
    df = (df.reindex(sorted(df.columns, key=attack_order.__getitem__), axis=1)
          .reset_index().fillna('-'))
    df['model_order'] = df.model_name.apply(model_order.__getitem__)
    df = df.sort_values(['model_order']).drop(columns='model_order')
    df['model_name'] = df.model_name.apply(formatted_names.__getitem__)    
    return df, df.to_latex(index=False)

# Ablation table

In [94]:
formatted_names = OrderedDict([
    ('cnn-mnist-relu', 'ReLU'),
    ('cnn-mnist-relog', 'ReLog'), 
    ('cnn-mnist-relog-maxout_2', '+ MaxOut (k=2)'),
    ('cnn-mnist-relog-minmaxout_4_2', '+ MinOut (k=4)'),
    ('cnn-mnist-relog-minmaxout_4_2-sigmoid_out', '+ Sigmoid output'),
    ('cnn-mnist-relog-spherical-minmaxout_4_2-sigmoid_out', '+ Spherical'),  
    ('cnn-mnist-relog-spherical-minmaxout_4_2-sigmoid_out-max_fit_l1_1', '+ MaxFit'),  
    ('cnn-mnist-relog-spherical-minmaxout_4_2-sigmoid_out-max_fit_l1_1-scrambling', '+ Scramble training'),  
])
model_order = {n: i for i, n in enumerate(formatted_names)}

In [95]:
used_attacks = [
    'FGM(L-2.0,eps=3.0)',
    'FGM(L-nan,eps=0.2)',
    'FGM(L-nan,eps=0.3)',
    'BIM(L-nan,eps=0.3)', 
    'C&W'
]
attack_order = {n: i for i, n in enumerate(used_attacks)}

In [96]:
ablation_table, latex = format_table(formatted_names, model_order, attack_order)

In [97]:
print(latex)

\begin{tabular}{llllll}
\toprule
       model\_name & FGM(L-2.0,eps=3.0) & FGM(L-nan,eps=0.2) & FGM(L-nan,eps=0.3) & BIM(L-nan,eps=0.3) &   C\&W \\
\midrule
             ReLU &               0.44 &               0.55 &               0.21 &               0.02 &  0.00 \\
            ReLog &               0.70 &               0.63 &               0.36 &               0.03 &  0.01 \\
   + MaxOut (k=2) &               0.77 &               0.70 &               0.54 &               0.09 &  0.01 \\
   + MinOut (k=4) &               0.79 &               0.83 &               0.68 &               0.21 &  0.02 \\
 + Sigmoid output &               0.84 &               0.87 &               0.74 &               0.31 &  0.19 \\
      + Spherical &               0.85 &               0.85 &               0.79 &               0.42 &     - \\
         + MaxFit &               0.84 &               0.87 &               0.79 &               0.49 &     - \\
     + Scrambling &               0.85 &            

# Regularization

In [143]:
formatted_names = OrderedDict([
    ('cnn-mnist-relu', 'ReLU'),
    ('cnn-mnist-relu-maxfit_l1_01_05', 'ReLU + MaxFit (l1)'),
    ('cnn-mnist-relu-maxfit_l2_01_05', 'ReLU + MaxFit (l2)'),
    ('cnn-mnist-relu-maxmargin_l1_01_05', 'ReLU + MaxMargin (l1)'),
    ('cnn-mnist-relu-maxmargin_l2_01_05', 'ReLU + MaxMargin (l2)'),
    ('cnn-mnist-relog-minmaxout_4_2-sigmoid_out', 'DNNP'),
    ('cnn-mnist-relog-minmaxout_4_2-sigmoid_out-max_margin', 'DNNP + MaxMargin' ),
    
])
order = {n: i for i, n in enumerate(formatted_names)}

In [147]:
ablation_table, latex = format_table(formatted_names, order)

In [148]:
print(latex)

\begin{tabular}{lllll}
\toprule
            model\_name &   C\&W & FGM(L-2.0,eps=1.5) & FGM(L-inf,eps=0.2) & FGM(L-inf,eps=0.3) \\
\midrule
                  ReLU &  0.00 &               0.82 &               0.44 &               0.09 \\
    ReLU + MaxFit (l1) &  0.00 &               0.77 &               0.57 &               0.21 \\
    ReLU + MaxFit (l2) &  0.00 &               0.79 &               0.53 &               0.19 \\
 ReLU + MaxMargin (l1) &  0.00 &               0.81 &               0.45 &               0.12 \\
 ReLU + MaxMargin (l2) &  0.00 &               0.81 &               0.44 &               0.08 \\
                  DNNP &  0.19 &               0.90 &               0.77 &               0.73 \\
      DNNP + MaxMargin &  0.31 &                  - &                  - &                  - \\
\bottomrule
\end{tabular}

